# HW1-1 Deep vs Shallow

### Package initialization

In [ ]:
import os
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"
TORCH_CUDA_ARCH_LIST="8.6"

import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.backends.cudnn as cudnn
import torchvision.transforms as transformtransforms

from torchvision import models
from torchsummary import summary
from torch.utils.data import Dataset, DataLoader
from torchvision.transforms import ToPILImage
from tqdm import tqdm

import copy
import math
import matplotlib.pyplot as plt
import numpy as np

Project_PATH = os.path.dirname(os.path.abspath('__file__'))
outputs_dir = Project_PATH + '/'
model_path = Project_PATH + '/save_models/'

### Device information

In [ ]:
print(torch.cuda.is_available())
print(torch.cuda.device_count())
print(torch.cuda.current_device())
device_default = torch.cuda.current_device()
torch.cuda.device(device_default)
print(torch.cuda.get_device_name(device_default))
device = torch.device("cuda")
print(torch.version.cuda)
print(torch.__version__)
print(torch.cuda.get_arch_list())

### Models

In [ ]:
'''
8 Layer DNN
'''
class DNN_8(nn.Module):
    def __init__(self):
        super(DNN_8, self, ).__init__()
        self.layer1 = nn.Sequential(nn.Linear(1, 5),nn.ReLU(True))
        self.layer2 = nn.Sequential(nn.Linear(5, 10),nn.ReLU(True))
        self.layer3 = nn.Sequential(nn.Linear(10, 10),nn.ReLU(True))
        self.layer4 = nn.Sequential(nn.Linear(10, 10),nn.ReLU(True))
        self.layer5 = nn.Sequential(nn.Linear(10, 10),nn.ReLU(True))
        self.layer6 = nn.Sequential(nn.Linear(10, 10),nn.ReLU(True))
        self.layer7 = nn.Sequential(nn.Linear(10, 5),nn.ReLU(True))
        self.layer8 = nn.Sequential(nn.Linear(5, 1))
    def forward(self, x):
        x = self.layer1(x) 
        x = self.layer2(x)    
        x = self.layer3(x)    
        x = self.layer4(x)    
        x = self.layer5(x)    
        x = self.layer6(x)    
        x = self.layer7(x)    
        x = self.layer8(x)    
        return x
    
device = torch.device("cuda")
Model_DNN_8 = DNN_8().to(device)
summary(Model_DNN_8, (1,1))

'''
5 Layer DNN
'''
class DNN_5(nn.Module):
    def __init__(self):
        super(DNN_5, self).__init__()
        self.layer1 = nn.Sequential(nn.Linear(1, 10),nn.ReLU(True))
        self.layer2 = nn.Sequential(nn.Linear(10, 18),nn.ReLU(True))
        self.layer3 = nn.Sequential(nn.Linear(18, 15),nn.ReLU(True))
        self.layer4 = nn.Sequential(nn.Linear(15, 4),nn.ReLU(True))
        self.layer5 = nn.Sequential(nn.Linear(4, 1))
    def forward(self, x):
        x = self.layer1(x) 
        x = self.layer2(x)    
        x = self.layer3(x)    
        x = self.layer4(x)    
        x = self.layer5(x)      
        return x
    
device = torch.device("cuda")
Model_DNN_5 = DNN_5().to(device)
summary(Model_DNN_5, (1,1))

'''
2 Layer DNN
'''
class DNN_2(nn.Module):
    def __init__(self):
        super(DNN_2, self).__init__()
        self.layer1 = nn.Sequential(nn.Linear(1, 190),nn.ReLU(True))
        self.layer2 = nn.Sequential(nn.Linear(190, 1))
    def forward(self, x):
        x = self.layer1(x) 
        x = self.layer2(x)         
        return x
    
device = torch.device("cuda")
Model_DNN_2 = DNN_2().to(device)
summary(Model_DNN_2, (1,1))


## HW1-1 Simulate a Function

### sin(5*np.pi*x)/(5*np.pi*x)

In [ ]:
x = torch.linspace(0,1,1000).unsqueeze(1)
y = torch.sin(5*np.pi*x)/(5*np.pi*x)

plt.figure()
plt.plot(x)
plt.plot(y)

### sgn(torch.sin(5*np.pi*x), 0)

In [ ]:
def sgn(x, y):
    h,w = list(x.size())
    Y = torch.rand(h,w)
    for i in range(h):
        if(x[i] > y): Y[i] = 1
        if(x[i] < y): Y[i] = -1
        if(x[i] == y): Y[i] = 0
    return Y

x = torch.linspace(0,1,1000).unsqueeze(1)
y = sgn(torch.sin(5*np.pi*x), 0)

plt.figure()
plt.plot(x)
plt.plot(y)

### HW1-1 Simulate a Function

In [ ]:
'''
Initiate data
'''
x = torch.linspace(0,1,1000).unsqueeze(1)
y = torch.sin(5*np.pi*x)/(5*np.pi*x)
y[0] = y[1]
func_1 = y

y = sgn(torch.sin(5*np.pi*x), 0)
func_2 = y

'''
Define train function
'''
def train(function,
          model_name,
          Epochs = 20000,
          Batch  = 1000,
          Data_workers = 0,
          LR = 0.0005):
    '''
    Initiate model
    '''
    torch.cuda.is_available()
    device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu') 
    Model = model_name().to(device)
    x = torch.linspace(0,1,1000).unsqueeze(1)
    x = x.to(device)
    y = function.to(device)
    '''
    loss & optimizer
    '''
    criterion = nn.MSELoss()
    optimizer = optim.Adam(Model.parameters(), lr=LR)
    scheduler = optim.lr_scheduler.StepLR(optimizer, step_size = 100, gamma = 0.8)
    '''
    Training
    '''
    trainloss_list = []
    lr_list = []
    for epoch in range(Epochs):
        Model.train()
        train_loss = 0.0
        y_pred = Model(x)
        loss = criterion(y_pred, y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        train_loss = loss.item()
        trainloss_list.append(train_loss)
        lr_list.append(optimizer.state_dict()['param_groups'][0]['lr'])
        if epoch >= Epochs//2:
            scheduler.step()

        if epoch % (Epochs//10) == 0:
            print('{}/{}, loss: {}'.format(epoch,Epochs,train_loss))
            
    return [Model,trainloss_list,lr_list]

In [ ]:
[Model_1_1,trainloss_1_1,lr_1_1] = train(func_1, DNN_8, Epochs=5000, Batch=1000, Data_workers=0, LR=0.005)
[Model_2_1,trainloss_2_1,lr_2_1] = train(func_1, DNN_5, Epochs=5000, Batch=1000, Data_workers=0, LR=0.005)
[Model_3_1,trainloss_3_1,lr_3_1] = train(func_1, DNN_2, Epochs=5000, Batch=1000, Data_workers=0, LR=0.005)

In [ ]:
torch.cuda.is_available()
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu') 
x = torch.linspace(0,1,1000).unsqueeze(1)
x = x.to(device)
y = func_1.to(device)
criterion = nn.MSELoss()

Model_1_1.eval()
Model_2_1.eval()
Model_3_1.eval()
with torch.no_grad():
    y_pred_1_1 = Model_1_1(x)
    y_pred_2_1 = Model_2_1(x)
    y_pred_3_1 = Model_3_1(x)
    testloss_1_1 = criterion(y_pred_1_1, y)
    testloss_2_1 = criterion(y_pred_2_1, y)
    testloss_3_1 = criterion(y_pred_3_1, y)
    print('Model_1_func_1 loss: {}'.format(testloss_1_1))
    print('Model_2_func_1 loss: {}'.format(testloss_2_1))
    print('Model_3_func_1 loss: {}'.format(testloss_3_1))

plt.figure(figsize=(20,10))
plt.plot(y.cpu().numpy(),label='Ref line')
plt.plot(y_pred_1_1.detach().cpu().numpy(),label='Model_1_1')
plt.plot(y_pred_2_1.detach().cpu().numpy(),label='Model_2_1')
plt.plot(y_pred_3_1.detach().cpu().numpy(),label='Model_3_1')
plt.xlabel('x',fontsize=20)
plt.ylabel('y',fontsize=20)
plt.title('sin(5*np.pi*x)/(5*np.pi*x)',fontsize=20)
plt.legend(fontsize=20)
plt.show()

plt.figure(figsize=(20,10))
plt.plot(trainloss_1_1, label='Model_1_1 loss')
plt.plot(trainloss_2_1, label='Model_2_1 loss')
plt.plot(trainloss_3_1, label='Model_3_1 loss')
plt.xlabel('epoch',fontsize=20)
plt.ylabel('loss',fontsize=20)
plt.title('Train loss',fontsize=20)
plt.legend(fontsize=20)
plt.show()

In [ ]:
[Model_1_2,trainloss_1_2,lr_1_2] = train(func_2, DNN_8, Epochs=5000, Batch=1000, Data_workers=0, LR=0.005)
[Model_2_2,trainloss_2_2,lr_2_2] = train(func_2, DNN_5, Epochs=5000, Batch=1000, Data_workers=0, LR=0.005)
[Model_3_2,trainloss_3_2,lr_3_2] = train(func_2, DNN_2, Epochs=5000, Batch=1000, Data_workers=0, LR=0.005)

In [ ]:
torch.cuda.is_available()
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu') 
x = torch.linspace(0,1,1000).unsqueeze(1)
x = x.to(device)
y = func_2.to(device)
criterion = nn.MSELoss()

Model_1_2.eval()
Model_2_2.eval()
Model_3_2.eval()
with torch.no_grad():
    y_pred_1_2 = Model_1_2(x)
    y_pred_2_2 = Model_2_2(x)
    y_pred_3_2 = Model_3_2(x)
    testloss_1_2 = criterion(y_pred_1_2, y)
    testloss_2_2 = criterion(y_pred_2_2, y)
    testloss_3_2 = criterion(y_pred_3_2, y)
    print('Model_1_func_2 loss: {}'.format(testloss_1_2))
    print('Model_2_func_2 loss: {}'.format(testloss_2_2))
    print('Model_3_func_2 loss: {}'.format(testloss_3_2))

plt.figure(figsize=(20,10))
plt.plot(y.cpu().numpy(),label='Ref line')
plt.plot(y_pred_1_2.detach().cpu().numpy(),label='Model_1_2')
plt.plot(y_pred_2_2.detach().cpu().numpy(),label='Model_2_2')
plt.plot(y_pred_3_2.detach().cpu().numpy(),label='Model_3_2')
plt.xlabel('x',fontsize=20)
plt.ylabel('y',fontsize=20)
plt.title('sgn(sin(5*np.pi*x))',fontsize=20)
plt.legend(fontsize=20)
plt.show()

plt.figure(figsize=(20,10))
plt.plot(trainloss_1_2, label='Model_1_2 loss')
plt.plot(trainloss_2_2, label='Model_2_2 loss')
plt.plot(trainloss_3_2, label='Model_3_2 loss')
plt.xlabel('epoch',fontsize=20)
plt.ylabel('loss',fontsize=20)
plt.title('Train loss',fontsize=20)
plt.legend(fontsize=20)
plt.show()

## HW1-1 Train on Actual Tasks

### Models

In [ ]:
class CNN_CIFAR_3(nn.Module):
    def __init__(self):
        super(CNN_CIFAR_3, self).__init__()
        self.layer1 = nn.Sequential(nn.Conv2d(3, 10, 3),nn.ReLU(True),nn.MaxPool2d(kernel_size=(2, 2), stride=2))
        self.layer2 = nn.Sequential(nn.Conv2d(10, 16, 3),nn.ReLU(True),nn.MaxPool2d(kernel_size=(2, 2), stride=2))
        self.layer3 = nn.Sequential(nn.Conv2d(16, 32, 3),nn.ReLU(True))
        self.layer4 = nn.Sequential(nn.Linear(32*4*4, 128),nn.ReLU(True))
        self.layer5 = nn.Sequential(nn.Linear(128, 32),nn.ReLU(True))
        self.layer6 = nn.Sequential(nn.Linear(32, 10))
    def forward(self, x):
        x = self.layer1(x) 
        x = self.layer2(x)    
        x = self.layer3(x) 
        x = x.view(x.size()[0], -1)
        x = self.layer4(x)    
        x = self.layer5(x)    
        x = self.layer6(x)    
        return x
    
device = torch.device("cuda")
Model = CNN_CIFAR_3().to(device)
summary(Model, input_size=(3,32,32))


class CNN_CIFAR_2(nn.Module):
    def __init__(self):
        super(CNN_CIFAR_2, self).__init__()
        self.layer1 = nn.Sequential(nn.Conv2d(3, 6, 5),nn.ReLU(True),nn.MaxPool2d(kernel_size=(2, 2), stride=2))
        self.layer2 = nn.Sequential(nn.Conv2d(6, 16, 5),nn.ReLU(True),nn.MaxPool2d(kernel_size=(2, 2), stride=2))
        self.layer3 = nn.Sequential(nn.Linear(16*5*5, 64),nn.ReLU(True))
        self.layer4 = nn.Sequential(nn.Linear(64, 32),nn.ReLU(True))
        self.layer5 = nn.Sequential(nn.Linear(32, 10))
    def forward(self, x):
        x = self.layer1(x) 
        x = self.layer2(x)  
        x = x.view(x.size()[0], -1)
        x = self.layer3(x) 
        x = self.layer4(x)    
        x = self.layer5(x)     
        return x
    
device = torch.device("cuda")
Model = CNN_CIFAR_2().to(device)
summary(Model, input_size=(3,32,32))


class CNN_CIFAR_1(nn.Module):
    def __init__(self):
        super(CNN_CIFAR_1, self).__init__()
        self.layer1 = nn.Sequential(nn.Conv2d(3, 16, 5),nn.ReLU(True),nn.MaxPool2d(kernel_size=(3, 3), stride=3))
        self.layer2 = nn.Sequential(nn.Linear(16*9*9, 64),nn.ReLU(True))
        self.layer3 = nn.Sequential(nn.Linear(64, 32),nn.ReLU(True))
        self.layer4 = nn.Sequential(nn.Linear(32, 10))
    def forward(self, x):
        x = self.layer1(x) 
        x = x.view(x.size()[0], -1)
        x = self.layer2(x)  
        x = self.layer3(x) 
        x = self.layer4(x)    
        return x
    
device = torch.device("cuda")
Model = CNN_CIFAR_1().to(device)
summary(Model, input_size=(3,32,32))


class DNN_MNIST_N(nn.Module):
    def __init__(self, in_dim, hidden_1, hidden_2, hidden_3, out_dim):
        super(DNN_MNIST_N, self).__init__()
        self.layer1 = nn.Sequential(nn.Linear(in_dim, hidden_1),nn.BatchNorm1d(hidden_1),nn.ReLU(True))
        self.layer2 = nn.Sequential(nn.Linear(hidden_1, hidden_2),nn.BatchNorm1d(hidden_2),nn.ReLU(True))
        self.layer3 = nn.Sequential(nn.Linear(hidden_2, hidden_3),nn.BatchNorm1d(hidden_3),nn.ReLU(True))
        self.layer4 = nn.Sequential(nn.Linear(hidden_3, out_dim))
    def forward(self, x):
        x = self.layer1(x) 
        x = self.layer2(x)    
        x = self.layer3(x)    
        x = self.layer4(x)
        return x
    
# device = torch.device("cuda")
# Model = DNN_MNIST_N(28*28,10,20,10,10).to(device)
# Model.eval()
# print('# of total parameters: ', sum(param.numel() for param in Model.parameters()))
# summary(Model, input_size=(1,28*28))


## CNN with CIFAR10

In [ ]:
import torch
import torchvision
import torchvision.transforms.functional as TF
from torch import nn
from torchvision import transforms
from torchvision.transforms import ToPILImage
from torch.utils.data import Dataset, DataLoader

import cv2
import random
import numpy as np
from PIL import Image
from glob import glob

'''
Define train function
'''
def train_CIFAR(model_name,
                Epochs = 100,
                Batch  = 2000,
                Data_workers = 0,
                LR = 0.1):
    '''
    Initiate data
    '''
    transform = transforms.Compose([transforms.ToTensor(),transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))])
    trainset = torchvision.datasets.CIFAR10(root='./data/',train=True,download=True,transform=transform)
    testset = torchvision.datasets.CIFAR10(root='./data/',train=False,download=True,transform=transform)
    trainloader = DataLoader(trainset, batch_size=Batch, shuffle=True, num_workers=Data_workers)
    testloader  = DataLoader(testset,  batch_size=Batch, shuffle=True, num_workers=Data_workers)
    print(trainset.classes)
    print(trainset.data.shape)
    print(testset.data.shape)
    '''
    Initiate model
    '''
    torch.cuda.is_available()
    device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu') 
    Model = model_name().to(device)
    '''
    loss & optimizer
    '''
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(Model.parameters(), lr=LR, momentum=0.9)
    scheduler = optim.lr_scheduler.StepLR(optimizer, step_size = 5, gamma = 0.8)
    '''
    Training
    '''
    trainloss_list = []
    testloss_list  = []
    accuracy_list  = []
    lr_list = []
   
    for epoch in range(Epochs):
        Model.train()
        train_loss = 0.0
#         with tqdm(total=(len(trainset) - len(trainset) % Batch)) as t:
#             t.set_description('epoch: {}/{}'.format(epoch+1, Epochs))
        for i, data in enumerate(trainloader):
            images, labels = data
            images = images.to(device)
            labels = labels.to(device)
            outputs = Model(images)
            loss = criterion(outputs, labels)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            train_loss += loss.item()
            total = (i+1)*Batch
#             t.set_postfix(loss='{:.6f}'.format(train_loss))
#             t.update(len(images))
        '''
        Evaluating
        '''
        Model.eval()
        with torch.no_grad():
            test_loss = 0
            correct = 0
            total = 0
            for data in testloader:
                images, labels = data
                images = images.to(device)
                labels = labels.to(device)
                outputs = Model(images)
                loss = criterion(outputs, labels)
                test_loss += loss.item()
                _, pred = torch.max(outputs.data, 1)
                correct += (pred == labels).cpu().sum()
                total += labels.size(0)
            total = len(testloader.dataset)
            accuracy = 100.0*correct/total
        '''
        Save loss
        '''
        scheduler.step()
        lr_list.append(optimizer.state_dict()['param_groups'][0]['lr'])
        trainloss_list.append(train_loss)
        testloss_list.append(test_loss)
        accuracy_list.append(accuracy)
        print('{}/{} Test set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%) lr={}'.format(
                epoch, Epochs,test_loss, correct, total, accuracy, lr_list[-1]))

    return [trainloss_list,
            testloss_list,
            accuracy_list,
            lr_list]


In [ ]:
[trainloss_CIFAR_3,testloss_CIFAR_3,accuracy_CIFAR_3,lr_CIFAR_3] = train_CIFAR(model_name=CNN_CIFAR_3,Epochs=100,Batch=2000,Data_workers=0,LR=0.1)
[trainloss_CIFAR_2,testloss_CIFAR_2,accuracy_CIFAR_2,lr_CIFAR_2] = train_CIFAR(model_name=CNN_CIFAR_3,Epochs=100,Batch=2000,Data_workers=0,LR=0.1)
[trainloss_CIFAR_1,testloss_CIFAR_1,accuracy_CIFAR_1,lr_CIFAR_1] = train_CIFAR(model_name=CNN_CIFAR_3,Epochs=100,Batch=2000,Data_workers=0,LR=0.1)

In [ ]:
'''
Plot loss & acc
'''
plt.figure(figsize=(20,10))
plt.plot(trainloss_CIFAR_3, label='CNN_3 loss')
plt.plot(trainloss_CIFAR_2, label='CNN_2 loss')
plt.plot(trainloss_CIFAR_1, label='CNN_1 loss')
plt.xlabel('epoch',fontsize=20)
plt.ylabel('loss',fontsize=20)
plt.title('Train loss',fontsize=20)
plt.legend(fontsize=20)
plt.show()

plt.figure(figsize=(20,10))
plt.plot(accuracy_CIFAR_3, label='CNN_3 accuracy')
plt.plot(accuracy_CIFAR_2, label='CNN_2 accuracy')
plt.plot(accuracy_CIFAR_1, label='CNN_1 accuracy')
plt.xlabel('epoch',fontsize=20)
plt.ylabel('accuracy',fontsize=20)
plt.title('Accuracy',fontsize=20)
plt.legend(fontsize=20)
plt.show()

## DNN with MNIST

In [ ]:
import torch
import torchvision
import torchvision.transforms.functional as TF
from torch import nn
from torchvision import transforms
from torchvision.transforms import ToPILImage
from torch.utils.data import Dataset, DataLoader

import cv2
import random
import numpy as np
from PIL import Image
from glob import glob

'''
Define train function
'''
def train_MNIST(model_name,
                Epochs = 50,
                Batch  = 2000,
                Data_workers = 0,
                LR = 0.1):
    '''
    Initiate data
    '''
#     transform = transforms.Compose([transforms.ToTensor(),transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))])
#     trainset = torchvision.datasets.CIFAR10(root='./data/',train=True,download=True,transform=transform)
#     testset = torchvision.datasets.CIFAR10(root='./data/',train=False,download=True,transform=transform)
    trainset = torchvision.datasets.MNIST(root='./data/',train=True,download=True,transform=transforms.ToTensor())
    testset = torchvision.datasets.MNIST(root='./data/',train=False,download=True,transform=transforms.ToTensor())
    trainloader = DataLoader(trainset, batch_size=Batch, shuffle=True, num_workers=Data_workers)
    testloader  = DataLoader(testset,  batch_size=Batch, shuffle=True, num_workers=Data_workers)
    print(trainset.classes)
    print(trainset.data.shape)
    print(testset.data.shape)
    '''
    Initiate model
    '''
    torch.cuda.is_available()
    device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu') 
    Model = model_name.to(device)
    '''
    loss & optimizer
    '''
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(Model.parameters(), lr=LR, momentum=0.9)
    scheduler = optim.lr_scheduler.StepLR(optimizer, step_size = 5, gamma = 0.8)
    '''
    Training
    '''
    trainloss_list = []
    testloss_list  = []
    accuracy_list  = []
    lr_list = []
   
    for epoch in range(Epochs):
        Model.train()
        train_loss = 0.0
#         with tqdm(total=(len(trainset) - len(trainset) % Batch)) as t:
#             t.set_description('epoch: {}/{}'.format(epoch+1, Epochs))
        for i, data in enumerate(trainloader):
            images, labels = data
#             images = images.to(device)
            images = (images.view(-1, 28*28)).to(device)
            labels = labels.to(device)
            outputs = Model(images)
            loss = criterion(outputs, labels)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            train_loss += loss.item()
            total = (i+1)*Batch
#             t.set_postfix(loss='{:.6f}'.format(train_loss))
#             t.update(len(images))
        '''
        Evaluating
        '''
        Model.eval()
        with torch.no_grad():
            test_loss = 0
            correct = 0
            total = 0
            for data in testloader:
                images, labels = data
#                 images = images.to(device)
                images = (images.view(-1, 28*28)).to(device)
                labels = labels.to(device)
                outputs = Model(images)
                loss = criterion(outputs, labels)
                test_loss += loss.item()
                _, pred = torch.max(outputs.data, 1)
                correct += (pred == labels).cpu().sum()
                total += labels.size(0)
            total = len(testloader.dataset)
            accuracy = 100.0*correct/total
        '''
        Save loss
        '''
        scheduler.step()
        lr_list.append(optimizer.state_dict()['param_groups'][0]['lr'])
        trainloss_list.append(train_loss)
        testloss_list.append(test_loss)
        accuracy_list.append(accuracy)
        print('{}/{} Test set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%) lr={}'.format(
                epoch, Epochs,test_loss, correct, total, accuracy, lr_list[-1]))

    return [trainloss_list,
            testloss_list,
            accuracy_list,
            lr_list]


In [ ]:
[trainloss_MNIST_3,testloss_MNIST_3,accuracy_MNIST_3,lr_MNIST_3] = train_MNIST(model_name=DNN_MNIST_N(28*28,1,2,1,10))
[trainloss_MNIST_2,testloss_MNIST_2,accuracy_MNIST_2,lr_MNIST_2] = train_MNIST(model_name=DNN_MNIST_N(28*28,2,4,2,10))
[trainloss_MNIST_1,testloss_MNIST_1,accuracy_MNIST_1,lr_MNIST_1] = train_MNIST(model_name=DNN_MNIST_N(28*28,4,8,4,10))

In [ ]:
'''
Plot loss & acc
'''
plt.figure(figsize=(20,10))
plt.plot(trainloss_MNIST_3, label='DNN_3 loss')
plt.plot(trainloss_MNIST_2, label='DNN_2 loss')
plt.plot(trainloss_MNIST_1, label='DNN_1 loss')
plt.xlabel('epoch',fontsize=20)
plt.ylabel('loss',fontsize=20)
plt.title('Train loss',fontsize=20)
plt.legend(fontsize=20)
plt.show()

plt.figure(figsize=(20,10))
plt.plot(accuracy_MNIST_3, label='DNN_3 accuracy')
plt.plot(accuracy_MNIST_2, label='DNN_2 accuracy')
plt.plot(accuracy_MNIST_1, label='DNN_1 accuracy')
plt.xlabel('epoch',fontsize=20)
plt.ylabel('accuracy',fontsize=20)
plt.title('Accuracy',fontsize=20)
plt.legend(fontsize=20)
plt.show()